# set up

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

In [ ]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   
    
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))  

# Load images

In [ ]:
import os
# Get a list of all the image files in the folder
image_folder = '/home/noahr/repos/personal_projects/img_to_svg/Images'
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.jpg')]

# Load model

In [ ]:
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth

In [ ]:
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_l_0b3195.pth

In [ ]:
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

In [1]:
!wget https://download.pytorch.org/models/retinanet_resnet50_fpn_coco-eeacb38b.pth

--2023-06-18 20:17:15--  https://download.pytorch.org/models/retinanet_resnet50_fpn_coco-eeacb38b.pth
Resolving download.pytorch.org (download.pytorch.org)... 99.86.178.43, 99.86.178.50, 99.86.178.33, ...
Connecting to download.pytorch.org (download.pytorch.org)|99.86.178.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 136595076 (130M) [binary/octet-stream]
Saving to: ‘retinanet_resnet50_fpn_coco-eeacb38b.pth.1’

retinanet_resnet50_ 100%[===================>] 130.27M  17.9MB/s    in 7.5s    

2023-06-18 20:17:22 (17.4 MB/s) - ‘retinanet_resnet50_fpn_coco-eeacb38b.pth.1’ saved [136595076/136595076]



In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor

sam_checkpoint = "sam_vit_l_0b3195.pth"
model_type = "vit_l"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

# Define paths for folders

In [ ]:
# Define the paths to the images and masks folders (Change these to other folders in your machine if you prefer that)
images_path = os.path.join(os.getcwd(), 'Images')
masks_path = os.path.join(os.getcwd(), 'masks')
cutouts_path = os.path.join(os.getcwd(), 'cutouts')
svg_path = os.path.join(os.getcwd(), 'svg')

# Generate coco masks

In [ ]:
from imageai.Detection import ObjectDetection
import sys
import os
import torch
import json
import cv2
import numpy as np
sys.path.append("..")
import pycocotools.mask as maskUtils
import os

# Loop through all the files in the images folder
for filename in os.listdir(images_path):
    # Check if a mask file already exists for this image
    mask_number = 0
    mask_filename = os.path.splitext(filename)[0] + '_mask_{}.json'.format(mask_number)
    while os.path.exists(os.path.join(masks_path, mask_filename)):
        mask_number += 1
        mask_filename = os.path.splitext(filename)[0] + '_mask_{}.json'.format(mask_number)
    if mask_number > 0:
        continue
    
    # Load the image
    image = cv2.imread(os.path.join(images_path, filename))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Load the object detection model
    detector = ObjectDetection()
    detector.setModelTypeAsRetinaNet()
    detector.setModelPath("/home/noahr/repos/personal_projects/img_to_svg/retinanet_resnet50_fpn_coco-eeacb38b.pth")
    detector.loadModel()
    
    # Detect humans in the image
    detections = detector.detectObjectsFromImage(input_image=os.path.join(images_path, filename), minimum_percentage_probability=50, extract_detected_objects=False)
    
    # Filter the detections to only include humans
    human_detections = [detection for detection in detections if detection["name"] == "person"]
    
    # Load the predictor model
    predictor = torch.hub.load('facebookresearch/detectron2:main', 'mask_rcnn', pretrained=True)
    predictor.eval()
    
    # Process each human detection
    for detection in human_detections:
        input_box = detection["box_points"]
        masks, _, _ = predictor.predict(
            point_coords=None,
            point_labels=None,
            box=np.array(input_box)[None, :],
            multimask_output=True,
        )
        for i, mask in enumerate(masks):
            # Convert the mask numpy array to a binary mask
            mask_binary = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.uint8)
            mask_binary[mask > 0] = 1
                
            # Convert the binary mask to a COCO RLE format
            mask_rle = maskUtils.encode(np.asfortranarray(mask_binary))
                
            # Extract the counts key from the mask RLE dictionary
            counts = mask_rle['counts']
                
            # Create a new dictionary with the required keys for COCO RLE format
            mask_coco_rle = {
                'size': [mask.shape[0], mask.shape[1]],
                'counts': counts.decode('utf-8'),
            }
                
            # Save the mask COCO RLE to a file
            mask_filename = os.path.splitext(filename)[0] + '_mask_{}.json'.format(i)
            with open(os.path.join(masks_path, mask_filename), 'w') as f:
                json.dump(mask_coco_rle, f)

# Create cutouts

In [ ]:
import os
import numpy as np
import pycocotools.mask as maskUtils
import json
import cv2
import matplotlib.pyplot as plt


# Loop through all the files in the images folder
for filename in os.listdir(images_path):
    # Load the image
    image = cv2.imread(os.path.join(images_path, filename))
    
    # Load the corresponding mask
    mask_number = 0
    cutout_number = 0
    mask_filename = os.path.splitext(filename)[0] + '_mask_{}.json'.format(mask_number)
    while os.path.exists(os.path.join(masks_path, mask_filename)):
        with open(os.path.join(masks_path, mask_filename), 'r') as f:
            rle_dict = json.load(f)
        size = rle_dict['size']
        counts = rle_dict['counts']
        mask_decoded = maskUtils.decode({'size': size, 'counts': counts})
        mask_binary = np.zeros((size[0], size[1]), dtype=np.uint8)
        mask_binary[mask_decoded > 0] = 1

        # Resize the mask to match the shape of the image
        mask_resized = cv2.resize(mask_decoded, (image.shape[1], image.shape[0]))

        # Extract the cutout from the image using the mask
        cutout = image * mask_resized[..., np.newaxis]

        # Create an alpha channel for the cutout image
        alpha = np.zeros(cutout.shape[:2], dtype=np.uint8)
        alpha[mask_resized > 0] = 255
        cutout = cv2.merge((cutout, alpha))

        # Crop the cutout image to the bounding rectangle
        x, y, w, h = cv2.boundingRect(mask_resized)
        cutout = cutout[y:y+h, x:x+w]

        # Save the cutout to the cutouts folder
        cutout_filename = os.path.splitext(filename)[0] + '_cutout_{}.png'.format(cutout_number)
        if os.path.exists(os.path.join(cutouts_path, cutout_filename)):
            os.remove(os.path.join(cutouts_path, cutout_filename))
        cv2.imwrite(os.path.join(cutouts_path, cutout_filename), cutout)
        
        # Delete the mask file
        os.remove(os.path.join(masks_path, mask_filename))
        
        # Increment the mask and cutout numbers
        mask_number += 1
        cutout_number += 1
        mask_filename = os.path.splitext(filename)[0] + '_mask_{}.json'.format(mask_number)

## Make svg from png

In [ ]:
import os
import subprocess
from PIL import Image

# Define the paths to the cutouts and output folders
cutouts_path = '/home/noahr/repos/personal_projects/img_to_svg/cutouts'
output_path = '/home/noahr/repos/personal_projects/img_to_svg/svg'

# Loop through all the files in the cutouts folder
for filename in os.listdir(cutouts_path):
    if filename.endswith('.png'):
        # Define the paths to the input and output files
        png_path = os.path.join(cutouts_path, filename)
        pbm_path = os.path.join(cutouts_path, os.path.splitext(filename)[0] + '.pbm')
        svg_filename = os.path.splitext(filename)[0] + '.svg'
        svg_path = os.path.join(output_path, svg_filename)

        # Convert the PNG to PBM format
        with Image.open(png_path) as im:
            im.convert('1').save(pbm_path)

        # Call potrace to convert the PBM to SVG
        subprocess.call(['potrace', '-s', pbm_path, '-o', svg_path])